<a href="https://colab.research.google.com/github/5starkarma/face_smoothing/blob/master/face_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/5starkarma/face-smoothing.git

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
% cd /content/drive/My\ Drive/Colab Notebooks/face-smoothing

/content/drive/My Drive/Colab Notebooks/face-smoothing


In [23]:
pip install -U PyYAML

Requirement already up-to-date: PyYAML in /usr/local/lib/python3.6/dist-packages (5.3.1)


In [117]:
! python3 infer.py --input '/content/drive/My Drive/Colab Notebooks/face-smoothing/data/images/mel1.mp4'

405 720
OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Face detected: [289, 57, 466, 318]
Face detected: [287, 55, 464, 316]
Face detected: [287, 51, 462, 315]
Face detected: [287, 49, 461, 314]
Face detected: [286, 48, 461, 311]
Face detected: [287, 47, 462, 310]
Face detected: [288, 46, 464, 306]
Face detected: [287, 49, 465, 305]
Face detected: [288, 49, 466, 308]
Face detected: [289, 51, 466, 310]
Face detected: [289, 52, 467, 311]
Face detected: [290, 53, 466, 314]
Face detected: [290, 53, 466, 317]
Face detected: [289, 55, 467, 314]
Face detected: [288, 56, 467, 310]
Face detected: [289, 56, 466, 310]
Face detected: [289, 55, 466, 306]
Face detected: [289, 56, 466, 302]
Face detected: [290, 55, 467, 300]
Face detected: [291, 55, 468, 300]
Face detected: [292, 55, 469, 301]
Face detected: [294, 54, 470, 305]
Face detected: [296, 56, 470, 306]
Face detected: [296, 

In [83]:
! rm -r /content/drive/My\ Drive/Colab\ Notebooks/face-smoothing/data/output